In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr
from sklearn.metrics import mean_absolute_error

In [2]:
yf.pdr_override()

In [3]:
data=pdr.get_data_yahoo("AAPL", start="2022-11-1", end="2022-11-15")
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-01 00:00:00-04:00,155.080002,155.449997,149.130005,150.649994,150.400497,80379300
2022-11-02 00:00:00-04:00,148.949997,152.169998,145.000000,145.029999,144.789810,93604600
2022-11-03 00:00:00-04:00,142.059998,142.800003,138.750000,138.880005,138.650009,97918500
2022-11-04 00:00:00-04:00,142.089996,142.669998,134.380005,138.380005,138.380005,140716700
2022-11-07 00:00:00-05:00,137.110001,139.149994,135.669998,138.919998,138.919998,83374600


In [4]:
Op=np.array(data["Open"])
Hi=np.array(data["High"])
Lo=np.array(data["Low"])
Cl=np.array(data["Close"])
Adj=np.array(data["Adj Close"])
V=np.array(data["Volume"])


In [5]:
input = np.column_stack((Op,Hi,Lo,V))
output = Adj

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(input,output, test_size=0.3)

In [8]:
from xgboost import XGBClassifier, plot_importance, XGBRegressor

In [9]:
XGBR=XGBRegressor(objective='reg:squarederror', n_estimators=5000, min_child_weight=2, learning_rate=0.05, max_depth=3, reg_alpha=3) #L2 #reg_alpha=5 (L1)

In [10]:
XGBR.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=2,
             missing=nan, monotone_constraints='()', n_estimators=5000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=3, reg_lambda=1, ...)

In [11]:
hit=float(0)
y_pred = XGBR.predict(X_test)
total=float(len(y_pred))
a=y_pred.tolist()
b=y_test.tolist()
y_pred = XGBR.predict(input)



In [12]:
XGBR.score(X_test,y_test)

0.6806234127291387

In [13]:
data2=pdr.get_data_yahoo("AAPL", start="2022-11-16", end="2022-11-18")
Op=np.array(data2["Open"])
Hi=np.array(data2["High"])
Lo=np.array(data2["Low"])
Cl=np.array(data2["Close"])
Adj=np.array(data2["Adj Close"])
V=np.array(data2["Volume"])

realvalue = Adj
print("input : %s , Predicted : %.3f " % (X_test,y_pred[9]))


mean_absolute_error(Adj,y_pred[:len(Adj)])

[*********************100%***********************]  1 of 1 completed
input : [[1.48970001e+02 1.50279999e+02 1.47429993e+02 7.33741000e+07]
 [1.55080002e+02 1.55449997e+02 1.49130005e+02 8.03793000e+07]
 [1.38500000e+02 1.38550003e+02 1.34589996e+02 7.49178000e+07]] , Predicted : 145.571 


4.183784484863281

In [14]:
print(y_pred[:len(Adj)])

[145.57121 145.57121]


In [15]:
print(Adj)

[148.78999329 150.72000122]


In [16]:
np.corrcoef(XGBR.predict(X_test),y_test)

array([[1.        , 0.99204304],
       [0.99204304, 1.        ]])